In [2]:
import pandas as pd

df = pd.read_csv('data/DataSetSamples/sample_April2022.csv')

df.head()

/tmp/ipykernel_14808/2472706645.py:3: DtypeWarning: Columns (24) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('data/DataSetSamples/sample_April2022.csv')


,Unnamed: 0.1,Unnamed: 0,Grid_ID,Datetime,extract_year,extract_month,extract_day,C1,C2,C3,...,E1,E2,E3,E4,E5,E6,E7,E8,E9,E10
0,0,0,272,2022-04-04 09:50,2022,4,4,21.28,0.00,18.51,...,9.77,0.69,154.48,40.0,3698.0,android:Total Traffic,0.0,6.10,36.0,0.0
1,1,1,274,2022-04-04 09:50,2022,4,4,328.93,14.90,281.93,...,0.00,108663.78,2156.72,13196477.0,150094.0,NaN,0.0,6.68,144.0,0.0
2,2,2,281,2022-04-04 09:50,2022,4,4,96.64,1.66,76.03,...,0.00,437.43,679.86,3062.0,4323.0,NaN,0.0,5.00,49.0,0.0
3,3,3,313,2022-04-04 09:50,2022,4,4,43.09,0.00,38.24,...,0.00,32.81,1180.00,1542.0,21049.0,android:Total Traffic,0.0,4.16,20.0,0.0
4,4,4,320,2022-04-04 09:50,2022,4,4,339.67,8.28,318.87,...,0.00,14995.39,6389.75,2189042.0,662496.0,android:Total Traffic;android:com.google.andro...,0.0,8.09,285.0,0.0
